# Technical Factor - Moving Average Convergence Divergence (MACD)

In [5]:
import dai
import pandas as pd

In [6]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [7]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),
data_factor AS (
    SELECT
        date,
        instrument,
        close,
        m_ta_ema(close, 13) - m_ta_ema(close, 26) AS DIF, 
        m_ta_ema(DIF, 9)                          AS DEA, 
        DIF - DEA                                 AS HIST,
    FROM data_base
),
data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(DIF > 0 AND m_lag(DIF, 1) <= 0, 1, 0) AS TRBY1,
        IF(DIF < 0 AND m_lag(DIF, 1) >= 0, 1, 0) AS TRSL1,
    FROM data_factor
),
data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(HIST > 0 AND m_lag(HIST, 1) <= 0, 1, 0) AS MTBY1,
        IF(HIST < 0 AND m_lag(HIST, 1) >= 0, 1, 0) AS MTSL1,
        IF(HIST > 0 AND (HIST - m_lag(HIST, 1)) > 0, 1, 0) AS MTBY2,
        IF(HIST < 0 AND (HIST - m_lag(HIST, 1)) < 0, 1, 0) AS MTSL2,
    FROM data_factor
),
data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(DIF > 0 AND HIST < 0 AND (HIST - m_lag(HIST, 1)) > 0, 1, 0) AS RVBY1,
        IF(DIF < 0 AND HIST > 0 AND (HIST - m_lag(HIST, 1)) < 0, 1, 0) AS RVSL1,
    FROM data_factor
),
data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(HIST, 1) <= 0 AND HIST > 0 AND (DIF - m_lag(DIF, 1)) > 0, 1, 0) AS BKBY1,
        IF(m_lag(HIST, 1) >= 0 AND HIST < 0 AND (DIF - m_lag(DIF, 1)) < 0, 1, 0) AS BKSL1,
    FROM data_factor
),
data_combined AS (
    SELECT
        date,
        instrument,
        DIF,
        DEA,
        HIST,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        MTBY2,
        MTSL2,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)
SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [8]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,DIF,DEA,HIST,TRBY1,TRSL1,MTBY1,MTSL1,MTBY2,MTSL2,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-02-26,600519.SH,37.618478,25.633749,11.984730,0,0,0,0,0,0,0,0,0,0
1,2025-02-27,600519.SH,46.963528,29.899705,17.063824,0,0,0,0,1,0,0,0,0,0
2,2025-02-28,600519.SH,62.416646,36.403093,26.013553,0,0,0,0,1,0,0,0,0,0
3,2025-03-03,600519.SH,66.305191,42.383512,23.921678,0,0,0,0,0,0,0,0,0,0
4,2025-03-04,600519.SH,59.215010,45.749812,13.465198,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,2025-12-25,600519.SH,-28.315796,-44.606624,16.290827,0,0,0,0,1,0,0,0,0,0
206,2025-12-26,600519.SH,-21.461290,-39.977557,18.516267,0,0,0,0,1,0,0,0,0,0
207,2025-12-29,600519.SH,-22.841754,-36.550396,13.708642,0,0,0,0,0,0,0,1,0,0
208,2025-12-30,600519.SH,-30.830451,-35.406407,4.575956,0,0,0,0,0,0,0,1,0,0
